In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd

df = pd.read_csv('data/cancer_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   gender                 276 non-null    int64
 1   age                    276 non-null    int64
 2   smoking                276 non-null    int64
 3   yellow fingers         276 non-null    int64
 4   anxiety                276 non-null    int64
 5   peer pressure          276 non-null    int64
 6   chronic disease        276 non-null    int64
 7   fatigue                276 non-null    int64
 8   allergy                276 non-null    int64
 9   wheezing               276 non-null    int64
 10  alcohol consuming      276 non-null    int64
 11  coughing               276 non-null    int64
 12  shortness of breath    276 non-null    int64
 13  swallowing difficulty  276 non-null    int64
 14  chest pain             276 non-null    int64
 15  lung cancer            276 non-null    i

In [3]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

In [4]:
from sdv.single_table import TVAESynthesizer

synthesizer = TVAESynthesizer(
    metadata,
    embedding_dim=256,
    compress_dims=(256, 256),
    decompress_dims=(256, 256),
    epochs=2500,
)

synthesizer.fit(df)

synthesizer.get_loss_values()

d:\Projects\Visual Studio Code\lung_cancer_analysis\.venv\Lib\site-packages\sdv\single_table\base.py:79: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,Epoch,Batch,Loss
0,0,0,26.172392
1,1,0,26.263220
2,2,0,24.831133
3,3,0,25.033331
4,4,0,24.134647
...,...,...,...
2495,2495,0,6.478133
2496,2496,0,6.650020
2497,2497,0,6.817426
2498,2498,0,6.890628


In [6]:
# synthesizer = TVAESynthesizer.load('synthesizer.pkl')
synthesizer.save('synthesizer.pkl')

In [5]:
from sdv.evaluation.single_table import evaluate_quality

synth_df = synthesizer.sample(82600, batch_size=500)
big_df = pd.concat([df, synth_df], ignore_index=True).drop_duplicates()
report = evaluate_quality(df, big_df, metadata)
big_df.info()

Sampling rows:   0%|          | 0/82600 [00:00<?, ?it/s]

Sampling rows: : 83000it [00:39, 2125.12it/s]                         


Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 120/120 [00:12<00:00,  9.93it/s]

Overall Score: 89.58%

Properties:
- Column Shapes: 96.62%
- Column Pair Trends: 82.53%
<class 'pandas.core.frame.DataFrame'>
Index: 30741 entries, 0 to 82874
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   gender                 30741 non-null  int64
 1   age                    30741 non-null  int64
 2   smoking                30741 non-null  int64
 3   yellow fingers         30741 non-null  int64
 4   anxiety                30741 non-null  int64
 5   peer pressure          30741 non-null  int64
 6   chronic disease        30741 non-null  int64
 7   fatigue                30741 non-null  int64
 8   allergy                30741 non-null  int64
 9   wheezing               30741 non-null  int64
 10  alcohol consuming      30741 non-null  int64
 11  coughing               30741 non-null  in

In [7]:
big_df.to_csv('data/big_cancer_data.csv', index=False)